In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from delta.tables import DeltaTable
import os

* clone the accounts table
* enable deletion vectors on it
* merge the increment into it

In [2]:
spark = (
    SparkSession
    .builder
    .appName('delta-III')
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/home/ubuntu/spark-3.5.3-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1e862291-92f6-4bf6-aa4e-54beeb183e92;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 169ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

In [5]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

questions = os.path.join(project_path, 'data/questions')
users_base_path = os.path.join(project_path, 'data/users_base')
users_increment_path = os.path.join(project_path, 'data/users_increment')
accounts_output_path = os.path.join(project_path, 'output/delta/accounts')

In [3]:
spark.sql('drop table if exists accounts')

DataFrame[]

In [6]:
spark.sql(f"""
    CREATE TABLE accounts
    USING DELTA
    LOCATION '{accounts_output_path}'
""")

DataFrame[]

In [7]:
spark.sql('ALTER TABLE accounts SET TBLPROPERTIES (delta.enableDeletionVectors = true)')

DataFrame[]

In [18]:
spark.sql('desc detail accounts').select('properties', 'minReaderVersion', 'minWriterVersion').show(n=100, truncate=100)

+-------------------------------------------------------------------------+----------------+----------------+
|                                                               properties|minReaderVersion|minWriterVersion|
+-------------------------------------------------------------------------+----------------+----------------+
|{delta.enableChangeDataFeed -> true, delta.enableDeletionVectors -> true}|               3|               7|
+-------------------------------------------------------------------------+----------------+----------------+



In [19]:
increment = spark.read.parquet(users_increment_path)

In [20]:
(
    DeltaTable.forName(spark, 'accounts')
    .alias('accounts')
    .merge(
        increment.alias('increment'),
        'accounts.user_id == increment.user_id'
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)